## Import libraries

In [1]:
import numpy as np
import pandas as pd
import pickle
import lightgbm as lgb
import glob
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
import time
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import datetime
from sklearn.linear_model import Ridge, LinearRegression
import uqer
from uqer import DataAPI   #优矿api
client = uqer.Client(token='18266a7c0ac9f8cdbe00f9b2ecb65f42316a5f78d9cc22ebabcbd923593356e4')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

UQER SDK的DataAPI模块版本由84.0.71升级到84.0.72
127807@wmcloud.com 账号登录成功


## Useful Functions

In [2]:
def ticekrToStr(job_sum):
    job_sum = job_sum[job_sum.ticker<700000]
    job_sum.loc[job_sum.ticker<10,'temp']='00000'
    job_sum.loc[(job_sum.ticker<100)&(job_sum.ticker>=10),'temp']='0000'
    job_sum.loc[(job_sum.ticker<1000)&(job_sum.ticker>=100),'temp']='000'
    job_sum.loc[(job_sum.ticker<10000)&(job_sum.ticker>=1000),'temp']='00'
    job_sum.loc[job_sum.temp==job_sum.temp,'ticker'] = job_sum[job_sum.temp==job_sum.temp]['temp']+job_sum[job_sum.temp==job_sum.temp]['ticker'].astype(str)
    del job_sum['temp']
    job_sum['ticker'] = job_sum['ticker'].astype(str)
    return job_sum

def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

class ExcludeExtreme(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.df_median = None
        self.df_standard = None

    def fit(self, df_columnList):
        self.df_median = df_columnList.median()
        self.df_standard = df_columnList.apply(lambda x: x - self.df_median[x.name]).abs().median()
        return self

    def scaller(self, x):
        self.di_max = self.df_median[x.name] + 5 * self.df_standard[x.name]
        self.di_min = self.df_median[x.name] - 5 * self.df_standard[x.name]
        x = x.apply(lambda v: self.di_min if v < self.di_min else v)
        x = x.apply(lambda v: self.di_max if v > self.di_max else v)
        return x

    def transform(self, df_columnList):
        df_columnList = df_columnList.apply(self.scaller)
        return df_columnList

## Alpha

* alphas_all里的alpha 2017-01-03 -- 2022-04-01

* data里的alpha 2022-03-01 -- 2022-06-14

* pct1_cal里的data 2022-04-20 -- 2022-07-28

In [31]:
alpha_list = []
for num in range(0, 66):
    alpha_list.append('alpha_{}'.format(num))

In [34]:
pd.read_csv('alphas_all/066_alpha_21.csv')

,ticker,tradeDate,066_alpha_21
0,1,2017-01-03,NaN
1,1,2017-01-04,NaN
2,1,2017-01-05,NaN
3,1,2017-01-06,NaN
4,1,2017-01-09,NaN
...,...,...,...
6049511,999,2022-03-28,NaN
6049512,999,2022-03-29,NaN
6049513,999,2022-03-30,NaN
6049514,999,2022-03-31,NaN


In [35]:
def get_data(file_loc, f_x, date, thresh):
    result_list = []
    for name in f_x:
        print(name)
        alpha = pd.read_csv(file_loc + name + '.csv')
        if alpha.columns.values.tolist()[0] == 'date':
            alpha = alpha.rename(columns={"date": "tradeDate"})
        if file_loc != 'alphas_all/066_':
            alpha = alpha.set_index(['tradeDate']).stack().reset_index().rename(columns={'level_1': 'ticker', 0: name})
        else:
            col_name = '066_'+name
            alpha = alpha.rename(columns={col_name: name})
        alpha['ticker'] = alpha['ticker'].astype(np.int64)
        if date != '':
            alpha = alpha[alpha.tradeDate == date]
        alpha = alpha.set_index(['ticker', 'tradeDate'])
        alpha = alpha.sort_index(ascending=True)
        result_list.append(alpha)
    result = pd.concat(result_list, axis=1)
    result = result.reset_index()
    result = result.dropna(thresh=thresh)
    return result

In [36]:
alphas1 = get_data('alphas_all/066_', alpha_list, '', 20)
alphas2 = get_data('data/', alpha_list, '', 20)
alphas3 = get_data('pct1_cal/每日预测/data/', alpha_list, '', 20)
display(alphas1)
display(alphas2)
display(alphas3)

alpha_0
alpha_1
alpha_2
alpha_3
alpha_4
alpha_5
alpha_6
alpha_7
alpha_8
alpha_9
alpha_10
alpha_11
alpha_12
alpha_13
alpha_14
alpha_15
alpha_16
alpha_17
alpha_18
alpha_19
alpha_20
alpha_21
alpha_22
alpha_23
alpha_24
alpha_25
alpha_26
alpha_27
alpha_28
alpha_29
alpha_30
alpha_31
alpha_32
alpha_33
alpha_34
alpha_35
alpha_36
alpha_37
alpha_38
alpha_39
alpha_40
alpha_41
alpha_42
alpha_43
alpha_44
alpha_45
alpha_46
alpha_47
alpha_48
alpha_49
alpha_50
alpha_51
alpha_52
alpha_53
alpha_54
alpha_55
alpha_56
alpha_57
alpha_58
alpha_59
alpha_60
alpha_61
alpha_62
alpha_63
alpha_64
alpha_65
alpha_0
alpha_1
alpha_2
alpha_3
alpha_4
alpha_5
alpha_6
alpha_7
alpha_8
alpha_9
alpha_10
alpha_11
alpha_12
alpha_13
alpha_14
alpha_15
alpha_16
alpha_17
alpha_18
alpha_19
alpha_20
alpha_21
alpha_22
alpha_23
alpha_24
alpha_25
alpha_26
alpha_27
alpha_28
alpha_29
alpha_30
alpha_31
alpha_32
alpha_33
alpha_34
alpha_35
alpha_36
alpha_37
alpha_38
alpha_39
alpha_40
alpha_41
alpha_42
alpha_43
alpha_44
alpha_45
alpha_46
alp

,ticker,tradeDate,alpha_0,alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,alpha_6,alpha_7,...,alpha_56,alpha_57,alpha_58,alpha_59,alpha_60,alpha_61,alpha_62,alpha_63,alpha_64,alpha_65
14,1,2017-01-23,0.505398,0.92600,0.7334,NaN,NaN,NaN,NaN,NaN,...,0.2325,NaN,NaN,NaN,0.521,0.521,NaN,0.521,NaN,0.5160
15,1,2017-01-24,0.503890,0.90800,0.8000,NaN,NaN,NaN,NaN,NaN,...,0.2345,NaN,NaN,NaN,0.521,0.521,NaN,0.521,NaN,0.5400
16,1,2017-01-25,0.505563,0.91850,0.8667,NaN,NaN,NaN,NaN,NaN,...,0.2444,NaN,NaN,NaN,0.521,0.521,NaN,0.521,NaN,0.5320
17,1,2017-01-26,0.503347,0.91100,0.9330,NaN,NaN,NaN,NaN,NaN,...,0.2462,NaN,NaN,NaN,0.521,0.521,NaN,0.521,NaN,0.5330
18,1,2017-02-03,0.503449,0.90970,0.8000,NaN,NaN,NaN,NaN,NaN,...,0.2480,NaN,NaN,NaN,0.521,0.521,NaN,0.521,NaN,0.6290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6049511,689009,2022-03-28,0.520114,0.99100,0.6000,0.3423,NaN,-1.252,NaN,0.5,...,0.7256,NaN,NaN,0.627543,0.501,0.498,0.7000,0.501,NaN,0.5835
6049512,689009,2022-03-29,0.457299,0.92040,0.6665,0.3572,NaN,-1.307,NaN,0.5,...,0.7230,NaN,NaN,0.072007,0.501,0.498,0.7666,0.501,NaN,0.0434
6049513,689009,2022-03-30,0.479480,0.07056,0.7334,0.3591,NaN,-1.340,NaN,0.5,...,0.7270,NaN,NaN,-4.882852,0.501,0.498,0.6665,0.501,NaN,0.5320
6049514,689009,2022-03-31,0.549231,0.05548,0.8000,0.3643,NaN,-1.341,NaN,0.5,...,0.7260,NaN,NaN,3.042670,0.501,0.498,0.5670,0.501,NaN,0.5273


,ticker,tradeDate,alpha_0,alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,alpha_6,alpha_7,...,alpha_56,alpha_57,alpha_58,alpha_59,alpha_60,alpha_61,alpha_62,alpha_63,alpha_64,alpha_65
0,1,2022-03-28,13.686417,0.804156,0.600000,NaN,0.511905,NaN,NaN,0.500109,...,0.725534,NaN,NaN,NaN,0.511905,0.511905,0.687500,0.511905,NaN,0.583392
1,1,2022-03-29,14.483306,0.850134,0.666667,NaN,0.511905,NaN,NaN,0.500109,...,0.723313,NaN,NaN,NaN,0.511905,0.511905,0.647059,0.511905,NaN,0.543409
2,1,2022-03-30,19.058333,0.768527,0.733333,NaN,0.511905,NaN,NaN,0.500109,...,0.727143,NaN,NaN,NaN,0.511905,0.511905,0.944444,0.511905,NaN,0.532361
3,1,2022-03-31,13.730159,0.768665,0.800000,NaN,0.511905,NaN,NaN,0.500109,...,0.725906,NaN,NaN,NaN,0.511905,0.511905,0.947368,0.511905,NaN,0.527214
4,1,2022-04-01,13.453121,0.719840,0.866667,NaN,0.511905,NaN,NaN,0.500109,...,0.727352,NaN,NaN,NaN,0.511905,0.511905,0.950000,0.511905,NaN,0.539820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238452,688787,2022-04-29,NaN,NaN,NaN,NaN,0.501730,NaN,NaN,0.500110,...,NaN,NaN,0.501730,NaN,0.510381,0.501724,0.700000,0.510381,NaN,NaN
238453,688799,2022-04-25,NaN,NaN,NaN,NaN,0.501189,NaN,NaN,0.500109,...,0.218785,NaN,0.501196,-105.170906,0.501202,0.501185,0.033333,0.501211,NaN,NaN
238454,689009,2022-04-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500109,...,NaN,NaN,NaN,NaN,0.501000,0.500996,0.038462,0.501004,NaN,NaN
238455,689009,2022-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500110,...,NaN,NaN,NaN,NaN,0.500996,0.500996,0.566667,0.500996,NaN,NaN


,ticker,tradeDate,alpha_0,alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,alpha_6,alpha_7,...,alpha_56,alpha_57,alpha_58,alpha_59,alpha_60,alpha_61,alpha_62,alpha_63,alpha_64,alpha_65
0,1,2022-05-27,165.614230,0.139064,0.000000,NaN,0.511905,NaN,0.509234,0.500111,...,0.728792,NaN,0.511905,NaN,0.511905,0.511905,0.080000,0.511905,NaN,0.511038
1,1,2022-05-30,391.314305,0.099046,0.000000,NaN,0.511905,NaN,0.509118,0.500111,...,NaN,NaN,0.511905,NaN,0.511905,0.511905,0.038462,0.511905,NaN,0.517547
2,1,2022-05-31,377.430375,0.128634,0.066667,NaN,0.511905,NaN,0.509003,0.500111,...,0.730888,NaN,0.511905,NaN,0.511905,0.511905,0.074074,0.511905,NaN,0.531304
3,1,2022-06-01,511.396318,0.137585,0.133333,NaN,0.511905,NaN,0.508772,0.500111,...,0.734889,NaN,0.511905,NaN,0.511905,0.511905,0.071429,0.511905,NaN,0.513975
4,1,2022-06-02,850.418869,0.152140,0.000000,NaN,0.511905,NaN,0.509003,0.500111,...,0.731743,NaN,0.511905,NaN,0.511905,0.511905,0.034483,0.511905,NaN,0.514942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247445,688778,2022-06-14,NaN,NaN,NaN,NaN,0.501179,NaN,NaN,0.500111,...,NaN,NaN,0.501179,NaN,0.501193,0.500000,0.466667,0.501193,NaN,NaN
247446,688779,2022-06-23,NaN,NaN,NaN,NaN,0.501180,NaN,NaN,0.500110,...,NaN,NaN,0.501179,NaN,0.501188,0.500000,0.266667,0.501188,NaN,NaN
247447,688786,2022-06-13,NaN,NaN,NaN,NaN,0.504310,NaN,NaN,0.500111,...,NaN,NaN,0.504310,NaN,0.504348,0.504310,0.466667,0.504348,NaN,NaN
247448,688787,2022-06-13,NaN,NaN,NaN,NaN,0.501730,NaN,NaN,0.500111,...,NaN,NaN,0.501730,NaN,0.503472,0.500000,0.000000,0.503472,NaN,NaN


In [37]:
alpha_data = pd.concat([alphas1, alphas2, alphas3], axis=0)
alpha_data = alpha_data.drop_duplicates(subset=['ticker', 'tradeDate'])
alpha_data = alpha_data.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)

In [38]:
alpha_data

,ticker,tradeDate,alpha_0,alpha_1,alpha_2,alpha_3,alpha_4,alpha_5,alpha_6,alpha_7,...,alpha_56,alpha_57,alpha_58,alpha_59,alpha_60,alpha_61,alpha_62,alpha_63,alpha_64,alpha_65
0,1,2017-01-23,0.505398,0.926000,0.733400,NaN,NaN,NaN,NaN,NaN,...,0.232500,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.516000
1,1,2017-01-24,0.503890,0.908000,0.800000,NaN,NaN,NaN,NaN,NaN,...,0.234500,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.540000
2,1,2017-01-25,0.505563,0.918500,0.866700,NaN,NaN,NaN,NaN,NaN,...,0.244400,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.532000
3,1,2017-01-26,0.503347,0.911000,0.933000,NaN,NaN,NaN,NaN,NaN,...,0.246200,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.533000
4,1,2017-02-03,0.503449,0.909700,0.800000,NaN,NaN,NaN,NaN,NaN,...,0.248000,NaN,NaN,NaN,0.521000,0.521000,NaN,0.521000,NaN,0.629000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983993,689009,2022-07-22,NaN,0.718109,NaN,0.581942,NaN,NaN,0.502263,0.500109,...,0.749190,NaN,NaN,-5.093013,0.500998,0.500996,0.933333,0.500998,15.0,NaN
4983994,689009,2022-07-25,0.532273,0.591823,0.800000,0.585736,NaN,NaN,0.502371,0.500109,...,0.752544,NaN,NaN,-14.398698,0.500998,0.500996,0.833333,0.500998,NaN,0.540234
4983995,689009,2022-07-26,0.514619,0.669616,0.866667,0.587814,NaN,NaN,0.502156,0.500109,...,0.750539,NaN,NaN,1.956526,0.500998,0.500996,0.833333,0.500998,NaN,0.515320
4983996,689009,2022-07-27,0.537030,0.745145,0.933333,0.591175,NaN,NaN,0.501940,0.500109,...,0.751079,NaN,NaN,7.059825,0.500998,0.500996,0.933333,0.500998,NaN,0.551398


In [39]:
alpha_data.to_csv('pct1_cal/每日预测/alpha_data.csv', index=False)

In [ ]:
path1 = 'raw_data/alphas_066'
filenames = glob.glob(path1 + '/*.csv')
result = pd.DataFrame()
for file in filenames[:5]:
    i = 1
    dic_name = file[9:19]
    print(dic_name)
    name = dic_name[7:] + '_' +file[20:-4]
    print(name)
    alpha = pd.read_csv(file)
    alpha = alpha.loc[alpha['date'] >= '2017-01-03']
    if dic_name == 'alphas_036':
        alpha = alpha.rename(columns = lambda x: x[2:] if (x != 'date') else 'tradeDate')
    else:
        alpha = alpha.rename(columns={"date": "tradeDate"})
    alpha = alpha.set_index(['tradeDate']).stack().reset_index().rename(columns={'level_1': 'ticker', 0: name})
    alpha['ticker'] = alpha['ticker'].astype(int)
    alpha = alpha.set_index(['ticker', 'tradeDate'])
    alpha = alpha.sort_index(ascending=True)
    if result.size == 0:
        print('result is empty')
        result = alpha.copy()
    else:
        result = pd.concat([result, alpha], axis=1)
    print('----- Finished FILE {}------'.format(file))
print(result)
#result.to_csv('alphas_066.csv', index=False)

## Label

In [3]:
file = 'label_vwap_pct1_full.csv'
name = file[6:-4]
print(name)
label = name + '_rank'
alpha = pd.read_csv('raw_data/' + file)
alpha

vwap_pct1_full


,tradeDate,1,2,4,5,6,7,8,9,10,...,900938,900940,900941,900942,900943,900945,900946,900947,900952,900955
0,2017-01-03,0.001201,0.014194,0.011332,-0.004571,0.006056,-0.002276,0.004372,-0.001039,0.005789,...,0.000000,-0.003807,0.002755,0.005181,0.001450,0.004910,0.016313,0.001992,0.000000,-0.003226
1,2017-01-04,-0.003708,-0.013234,-0.012942,0.013201,0.022855,-0.024062,0.007963,-0.004350,-0.013940,...,-0.003110,0.001274,-0.006868,0.001473,-0.005793,-0.003257,-0.028892,-0.001988,-0.003774,-0.011327
2,2017-01-05,0.000438,-0.003956,-0.028638,-0.002549,-0.007282,-0.019864,-0.002528,0.002184,0.004280,...,-0.001560,-0.001272,0.001383,-0.000735,-0.008740,-0.001634,0.028099,0.000000,0.003788,-0.003273
3,2017-01-06,0.001204,-0.002664,0.011059,-0.001278,-0.014771,-0.013392,0.001690,0.008529,-0.000904,...,-0.001563,-0.006369,0.001381,-0.000736,0.012491,0.014730,-0.001608,0.001992,-0.001887,-0.003284
4,2017-01-09,-0.000328,-0.005390,-0.015695,-0.014501,0.003774,0.004833,-0.013388,-0.019921,-0.024560,...,-0.059468,-0.005128,-0.005517,-0.002209,-0.010160,-0.008065,-0.016103,-0.003976,0.003781,-0.004942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,2022-04-21,-0.047243,-0.024562,-0.075233,-0.018639,-0.056290,-0.018583,-0.030446,-0.072077,-0.059444,...,0.012448,-0.003175,-0.028640,-0.011034,-0.028825,0.040816,-0.053812,-0.016194,-0.011811,0.058824
1288,2022-04-22,-0.029172,-0.010072,-0.063486,-0.056095,-0.021734,-0.053402,-0.035825,-0.053224,-0.068637,...,-0.040984,0.000000,-0.014742,-0.001395,-0.015982,0.004902,-0.014218,-0.004115,0.003984,0.009259
1289,2022-04-25,0.045377,-0.042534,-0.032217,-0.054750,-0.024438,-0.009689,-0.009174,0.016577,-0.008391,...,0.000000,-0.003185,0.004988,-0.002793,0.000000,-0.004878,0.024038,0.008264,-0.007937,-0.073394
1290,2022-04-26,0.015053,0.014132,-0.060514,-0.049505,0.032389,0.049235,0.007407,0.030722,0.002208,...,0.000000,0.012780,0.007444,0.005602,0.013921,0.024510,0.009390,-0.004098,-0.008000,0.019802


In [4]:
alpha = alpha.loc[alpha['tradeDate'] >= '2017-01-03']
#alpha = alpha.rename(columns = lambda x: x[:-5] if (x != 'date') else 'tradeDate')
alpha = alpha.set_index(['tradeDate']).stack().reset_index().rename(columns={'level_1': 'ticker', 0: name})
alpha['ticker'] = alpha['ticker'].astype(int)
alpha = alpha.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
alpha[label] = alpha.groupby('tradeDate')[name].rank(pct = True)
alpha.to_csv('labels/{}.csv'.format(name), index=False)
print('----- Finished FILE {}------'.format(file))
alpha

----- Finished FILE label_vwap_pct1_full.csv------


,tradeDate,ticker,vwap_pct1_full,vwap_pct1_full_rank
0,2017-01-03,1,0.001201,0.412047
1,2017-01-04,1,-0.003708,0.558080
2,2017-01-05,1,0.000438,0.619885
3,2017-01-06,1,0.001204,0.506232
4,2017-01-09,1,-0.000328,0.751037
...,...,...,...,...
5014493,2022-04-21,900955,0.058824,0.989548
5014494,2022-04-22,900955,0.009259,0.952351
5014495,2022-04-25,900955,-0.073394,0.059131
5014496,2022-04-26,900955,0.019802,0.603679


## OutData

In [ ]:
def get_target(ticker_name):
    # get close price and market value
    outData = DataAPI.MktEqudGet(ticker=ticker_name, beginDate='2017-07-25', endDate='2022-04-05', pandas="1")
    outData = outData[['ticker', 'tradeDate', 'closePrice', 'marketValue']]
    # get industry
    industry_data = DataAPI.EquIndustryGet(ticker=ticker_name, pandas="1")
    industry_name = industry_data.at[0, 'industryName1']
    # get listDate
    list_date = DataAPI.EquGet(ticker=ticker_name ,pandas="1")
    list_date = list_date.at[0, 'listDate']
    # combine data
    outData['industry'] = industry_name
    outData['listData'] = list_date
    outData = outData.sort_values(by=['tradeDate']).reset_index(drop=True)
    return outData[['ticker', 'tradeDate', 'marketValue', 'industry', 'listData']]

print('load data')
data = pd.read_csv('pct1_cal/1alter_alphas_066.csv', usecols=['ticker'])
print(data)
strData = ticekrToStr(data)
tickers = strData.ticker.unique()
tickers = tickers.tolist()

df_merge = get_target(tickers)
print('sort values')
df_merge = df_merge.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
print(df_merge)
df_merge.to_csv('pct1_cal/3uqer_data.csv', index = False)

df_merge.industry = df_merge.industry.fillna('其他')
industry_to_number = {}
for i, v in enumerate(df_merge.industry.sort_values().unique()):
    industry_to_number[v] = i+1
print(industry_to_number)
df_merge.industry = df_merge.industry.map(industry_to_number)
df_idst = pd.get_dummies(df_merge.industry, prefix='idst')
df = df_merge.merge(df_idst, left_index=True, right_index=True)
df['log_marketValue'] = df.marketValue.apply(np.log)
df.to_csv('pct1_cal/4uqer_idst_log.csv', index = False)

## Combine Data

In [ ]:
print('========LOADING AND COMBINING ALTER DATA========')
alpha = pd.read_csv('pct1_cal/alphas_066.csv')
print(alpha.shape)
print('========COMPLETE LOADING AND COMBINING ALTER DATA========')

print('========LOADING ALTER DATA========')
alter = pd.read_csv('raw_data/alternative_factors.csv')
alter = alter.iloc[:, 1:]
print(alter.shape)
print('========COMPLETE LOADING ALTER DATA========')

print('========COMBINING ALTER ALPHA DATA========')
alter = alter.merge(alpha, on=['ticker', 'tradeDate'], how='left')
del alpha
print(alter.shape)
print('========COMPLETE COMBINING ALTER ALPHA DATA========')

print('========REALLOCATE ALTER ALPHA DATA========')
alter = alter.dropna(thresh = 87)
alter = reduce_mem_usage(alter)
print(alter)
f_x_066 = alter.columns.values.tolist()[2:]
print(f_x_066)
pickle.dump(f_x_066, open("pct1_cal/f_x_066", "wb"))
print('store alphas 222')
alter.to_csv('pct1_cal/1alter_alphas_066.csv', index = False)
print('========COMPLETE STORING ALTER ALPHAS 066========')

#alter = pd.read_csv('pct1_cal/1alter_alphas_066.csv')

print('========LOADING UQER DATA========')
uqer_data = pd.read_csv('pct1_cal/4uqer_idst_log.csv')
print(uqer_data.shape)
print('========COMPLETE LOADING UQER DATA========')

print('========COMBINE ALTER ALPHA UQER DATA========')
alter = alter.merge(uqer_data, on=['ticker', 'tradeDate'], how='left')
alter = alter[~alter.log_marketValue.isnull()]
print(alter.shape)
del uqer_data
alter = reduce_mem_usage(alter)
print('========COMPLETE COMBINE ALTER ALPHA UQER DATA========')

print('get column lists')
f_index = ['ticker', 'tradeDate']
f_industry = pickle.load(open("pct1_cal/f_industry", "rb"))
f_x = pickle.load(open("pct1_cal/f_x_066", "rb"))
label_list = ['askbid_pct1_rank', 'openclose_pct1_rank']

print('========COMBINING LABEL DATA========')
path2 = 'labels'
filenames = glob.glob(path2 + '/*pct1.csv')
for file in filenames:
    print(file)
    label = pd.read_csv(file)
    label_name = label.columns.values.tolist()[-1]
    label = label[['ticker', 'tradeDate', label_name]]
    alter = alter.merge(label, on=['ticker', 'tradeDate'], how='left')
    del label
alter = reduce_mem_usage(alter)
print(alter.shape)
print('========COMPLETE COMBINING LABEL DATA========')

print('store raw data')
alter.to_csv('pct1_cal/alter_idst_alphas_066_labels_raw.csv', index = False)

## DealWithData

In [ ]:
df = pd.read_csv('pct1_cal/alter_idst_alphas_066_labels_raw2.csv')
print(df)

print('get column lists')
f_index = ['ticker', 'tradeDate']
f_industry = pickle.load(open("pct1_cal/f_industry", "rb"))
f_x = pickle.load(open("pct1_cal/f_x_066", "rb"))
label_list = ['openclose_pct1'] #['PCT5_rank', 'PCT2_rank', 'openclose_pct1_rank', 'askbid_pct1_rank']

In [13]:
# %% a list of classes
## exclude extreme values
class ExcludeExtreme(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.df_median = None
        self.df_standard = None

    def fit(self, df_columnList):
        self.df_median = df_columnList.median()
        self.df_standard = df_columnList.apply(lambda x: x - self.df_median[x.name]).abs().median()
        return self

    def scaller(self, x):
        self.di_max = self.df_median[x.name] + 5 * self.df_standard[x.name]
        self.di_min = self.df_median[x.name] - 5 * self.df_standard[x.name]
        x = x.apply(lambda v: self.di_min if v < self.di_min else v)
        x = x.apply(lambda v: self.di_max if v > self.di_max else v)
        return x

    def transform(self, df_columnList):
        df_columnList = df_columnList.apply(self.scaller)
        return df_columnList

## fill empty values
class FillEmpty(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.df_mean_industry = None

    def fit(self, df_name, f_x = f_x, group_name='industry'):
        self.df_mean_industry = df_name.groupby(group_name).mean()[f_x]
        self.df_mean_industry = self.df_mean_industry.fillna(self.df_mean_industry.mean())
        self.df_mean_industry.columns = [x + '_mean' for x in self.df_mean_industry.columns]
        self.df_mean_industry = self.df_mean_industry.reset_index()
        self.df_mean_industry = self.df_mean_industry.fillna(0)  # 可以删除
        return self

    def transform(self, df_name, f_x = f_x, group_name='industry'):
        df_name_mean = df_name.merge(self.df_mean_industry, on=group_name, how='left')
        df_name_mean[f_x] = df_name_mean[f_x].apply(lambda x: x.fillna(df_name_mean[x.name + '_mean']))
        df_name = df_name_mean[df_name.columns]
        return df_name

## neutralize industry
class Neutralization(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.models = {}

    def regr_fit(self, X, y):
        self.models[y.name] = linear_model.LinearRegression().fit(X, y)
        return

    def regr_pred(self, X, y):
        pred = self.models[y.name].predict(X)
        return y - pred

    def fit(self, df_name, f_x = f_x, f_idst = f_industry):
        X = df_name[['log_marketValue'] + f_idst]
        df_name[f_x].apply(lambda y: self.regr_fit(X, y))
        return self

    def transform(self, df_name, f_x = f_x, f_idst = f_industry):
        X = df_name[['log_marketValue'] + f_idst]
        df_name[f_x] = df_name[f_x].apply(lambda y: self.regr_pred(X, y))
        return df_name

NameError: name 'f_x' is not defined

In [ ]:
dates = df.tradeDate.sort_values().unique()
timediff = pd.Timedelta(100,unit='d')

def run_df(date):
#for date in dates:
    print('----- DATE {}------'.format(date))
    df2 = df[df.tradeDate == date]
    print('处理前无效值：', df2[label_list].isnull().any().sum())
    df2 = df2[~df2.log_marketValue.isnull()]
    #df2 = df2[~df2.PCT2_rank.isnull()]
    #df2 = df2[~df2.PCT5_rank.isnull()]
    #df2 = df2[~df2.askbid_pct1_rank.isnull()]
    df2 = df2[~df2.openclose_pct1.isnull()]
    print('处理后无效值：', df2[label_list].isnull().any().sum())

    # 中位数去极值
    print('exclude extreme values')
    mevtransformer = ExcludeExtreme()
    mevtransformer.fit(df2[f_x])
    df2[f_x] = mevtransformer.transform(df2[f_x])

    # 缺失值处理
    print('deal with null values')
    gvfiller = FillEmpty()
    gvfiller = gvfiller.fit(df2)
    df2 = gvfiller.transform(df2)
    print('缺失值：', df2[f_x].isnull().any().sum())

    # 行业中性处理
    print('do neutralization')
    idst_neutral = Neutralization()
    idst_neutral = idst_neutral.fit(df2)
    df2 = idst_neutral.transform(df2)

    # 标准化处理
    print('do standardization')
    scaler = StandardScaler()
    df2[f_x] = scaler.fit_transform(df2[f_x])

    df2 = df2[f_index + f_x + label_list + ['listData']]
    #df1 = df1.append(df2)
    return df2
    #df2.to_csv('train_test/{}_data.csv'.format(date), index = False)

print('PARALLEL')
start = time.time()
parallel_obj = Parallel(n_jobs=48)(delayed(run_df)(date) for date in dates)
print(f'耗时:{time.time() - start}')
df1 = pd.concat(parallel_obj)
print('sort values')
df1 = df1.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
print('reduce memory')
df1 = reduce_mem_usage(df1)
print('store data')
df1.to_csv('pct1_cal/modified_alter_alphas_066_labels2.csv', index = False)

## Research

In [3]:
####========需要修改的全局参数========####
print('========LOADING DATA========')
df1 = pd.read_csv('pct1_cal/modified_alter_alphas_066_labels.csv')
label = pd.read_csv('labels/openclose_pct1_full.csv')
print('========COMPLETE LOADING DATA========')

========LOADING DATA========
========COMPLETE LOADING DATA========


In [4]:
df1 = df1.merge(label, on=['ticker', 'tradeDate'], how='left')
df1 = df1[~df1.openclose_pct1_full_rank.isnull()]
df1 = df1[~df1.openclose_pct1_full.isnull()]
df1 = df1[~df1.openclose_pct1_rank.isnull()]
df1 = df1[~df1.openclose_pct1.isnull()]

In [5]:
f_x_221 = pickle.load(open('pct1_cal/每日预测/f_x_221', 'rb'))
f_x = ['volume', 'cachgPct', 'thecommittee', 'askVolume1',
       'bidVolume1', 'caQrr', 'caTr', 'OCVP1',
       'Open/vwap-1', 'Gap']
f_x.extend(f_x_221)

In [6]:
label_list = ['PCT5_rank', 'PCT2_rank', 'openclose_pct1_rank', 'openclose_pct1', 
'openclose_pct1_full', 'openclose_pct1_full_rank', 
'vwap_pct1_full', 'vwap_pct1_full_rank', 'askbid_pct1_rank', 'askbid_pct1']
f_y_list =  label_list[2:6] # 'pct_dmean'
#f_x = pickle.load(open("pct1_cal/f_x_066", "rb"))

#e: extra, m: filted
data_source = 'full_Alter_066_221_extra'
file_location = 'pct1_cal'
file_type = 'full'
model_list = ['LinearRegression', 'RidgeR', 'DecisionTreeR', 'XGBoostR', 'LGBMRegressor']
model_name = model_list[4]
target_types = ['r', 'c'] # 分类问题还是回归问题 r 回归问题 c 分类问题
target_type = target_types[0]
####========需要修改的全局参数========####

In [18]:
dates = df1.tradeDate.sort_values().unique()
epoch_ts = list(dates)
f_index = ['ticker', 'tradeDate']
#result_name = '{}_{}_{}_{}'.format(data_source, model_name, f_y, target_type)
#print(result_name)

update = 22 # 训练长度：22天
train_si = epoch_ts.index('2017-01-03') # included. '2017-01-03'
train_ei = epoch_ts.index('2019-01-02') # excluded. '2018-12-28'
test_si = epoch_ts.index('2019-01-02') # included. '2019-01-02'
test_ei = epoch_ts.index('2019-02-01') # excluded. '2019-01-31' '2019-02-01'
test_fi = len(epoch_ts) - 1 # excluded. '2019-01-16'

# number of epochs，循环次数
num_epoch = round((test_fi - test_ei) / update)
epoch_range = range(0, num_epoch + 1)
timediff = pd.Timedelta(100,unit='d')

In [8]:
def load_lrlist(lr_max,lr_min,num_rounds):
    lrlist = [lr_max+(lr_min-lr_max)*(np.log(i)/np.log(num_rounds)) for i in range(1,num_rounds+1)]
    return lrlist

In [9]:
lr_max = 0.05
lr_min = 0.02
n_jobs = 20

In [10]:
#gbm params
params =  {
    'boosting_type' : 'gbdt',
    'num_leaves' : 31,
    'feature_fraction' : 0.8,
    'bagging_fraction' : 0.8,
    'lambda_l1':1,
    'lambda_l2':10,
    'max_bin' : 64,
    'num_boost_round': 200,
    'learning_rate' : 0.04,
    'min_data_in_leaf':10,
    'num_threads': n_jobs
}
params['objective'] = 'regression'
params['metric'] = {'l2', 'auc'}
lrlist = load_lrlist(lr_max,lr_min,params['num_boost_round'])
params['lrlist'] = lrlist

## Test Parameters

In [ ]:
epoch_t_train = epoch_ts[train_si + 34*22: train_ei + 34*22]
epoch_t_test = epoch_ts[test_si + 34*22: test_ei + 34*22]
df_train = df1[df1.tradeDate.apply(lambda x: x in epoch_t_train)].reset_index(drop=True)
df_test = df1[df1.tradeDate.apply(lambda x: x in epoch_t_test)].reset_index(drop=True)
print('预测时间：', epoch_t_test)
print('数据大小：', df_train.shape, df_test.shape)
# 数据筛选 删除上市100天以内的
a = pd.to_datetime(df_train.tradeDate)
b = pd.to_datetime(df_train.listData)
df_train = df_train[a-b > timediff]
a = pd.to_datetime(df_test.tradeDate)
b = pd.to_datetime(df_test.listData)
df_test = df_test[a-b > timediff]

x_train31 = df_train[f_x].values
x_test31 = df_test[f_x].values
print('处理后x:', x_train31.shape, x_test31.shape)

# 获得y
y_train31 = df_train[f_y].copy()
y_test31 = df_test[f_y].copy()
print('处理后y:', y_train31.shape, y_test31.shape)

In [ ]:
import optuna

In [ ]:
def objective(trial):
    params = {
        #'alpha': trial.suggest_float('alpha', 0, 0.1) # 0.0005 0.0744
        #'splitter': trial.suggest_categorical('splitter', ['random', 'best']),
        #'criterion': trial.suggest_categorical('criterion', ['friedman_mse']),
        "n_estimators": trial.suggest_int("n_estimators", 100, 300, step=50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, step=0.01),
        "subsample": trial.suggest_float("subsample", 0.7, 1, step=0.1),
        "max_depth": trial.suggest_int("max_depth", 3, 6),
        'subsample_freq' : trial.suggest_int('subsample_freq', 2, 8),
        'reg_lambda' :trial.suggest_int('reg_lambda', 0, 200, step=20),
        'num_leaves' : trial.suggest_int('num_leaves', 10, 80, step=10),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200, step=20),
        'max_bin': trial.suggest_int('max_bin', 20, 260, step=20)
        #'min_samples_leaf': trial.suggest_int('min_samples_leaf', 20, 60)
    }
    # 'n_estimators': [20], 'max_depth': [3], 'max_features': [10], 'subsample': [1.0]},
    # 'splitter': ['random'], 'criterion': ['friedman_mse'], 'max_depth': [6], 'min_samples_leaf': [41], 'min_impurity_decrease': [0.5]
    cv_scores = np.empty(5)

    model = lgb.LGBMRegressor(**params)
    model.fit(x_train13, y_train13)
    y_pred13 = model.predict(x_test13)
    df_result = pd.DataFrame({'y' : y_test13, 'y_pred': y_pred13})
    IC13 = df_result[['y', 'y_pred']].corr().iloc[0,1]
    print(IC13)
    cv_scores[0] = IC13

    model = lgb.LGBMRegressor(**params)
    model.fit(x_train14, y_train14)
    y_pred14 = model.predict(x_test14)
    df_result = pd.DataFrame({'y' : y_test14, 'y_pred': y_pred14})
    IC14 = df_result[['y', 'y_pred']].corr().iloc[0,1]
    print(IC14)
    cv_scores[1] = IC14

    model = lgb.LGBMRegressor(**params)
    model.fit(x_train19, y_train19)
    y_pred19 = model.predict(x_test19)
    df_result = pd.DataFrame({'y' : y_test19, 'y_pred': y_pred19})
    IC19 = df_result[['y', 'y_pred']].corr().iloc[0,1]
    print(IC19)
    cv_scores[2] = IC19

    model = lgb.LGBMRegressor(**params)
    model.fit(x_train21, y_train21)
    y_pred21 = model.predict(x_test21)
    df_result = pd.DataFrame({'y' : y_test21, 'y_pred': y_pred21})
    IC21 = df_result[['y', 'y_pred']].corr().iloc[0,1]
    print(IC21)
    cv_scores[3] = IC21

    model = lgb.LGBMRegressor(**params)
    model.fit(x_train31, y_train31)
    y_pred31 = model.predict(x_test31)
    df_result = pd.DataFrame({'y' : y_test31, 'y_pred': y_pred31})
    IC31 = df_result[['y', 'y_pred']].corr().iloc[0,1]
    print(IC31)
    cv_scores[4] = IC31

    return np.mean(cv_scores)

    """
    cv = TimeSeriesSplit(n_splits=5, test_size = 22)

    cv_scores = np.empty(5)

    # LGBM
    for idx, (train_idx, test_idx) in enumerate(cv.split(df_train[f_x], df_train[f_y])):
        x_cv_train, x_cv_test = df_train[f_x].iloc[train_idx], df_train[f_x].iloc[test_idx]
        y_cv_train, y_cv_test = df_train[f_y][train_idx], df_train[f_y][test_idx]

        model = lgb.LGBMRegressor(**params)
        model.fit(x_cv_train, y_cv_train)
        y_pred = model.predict(x_cv_test)

        #df_result = df_test[f_index].copy()
        #df_result['y'] = y_test
        #df_result['y_pred'] = y_pred
        # 获得结果
        df_result = pd.DataFrame({'y' : y_cv_test, 'y_pred': y_pred})
        #IC = df_result[['y', 'y_pred']].corr().iloc[0,1]
        cv_scores[idx] = df_result[['y', 'y_pred']].corr().iloc[0,1]
        print(cv_scores[idx])
    return np.mean(cv_scores)
    """

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
trial = study.best_params()
pickle.dump(trial, open('pct1_cal/trial', 'wb'))

## 2

In [11]:
alpha_list = []
for num in range(0, 66):
    alpha_list.append('alpha_{}'.format(num))
f_x.extend(alpha_list)

In [15]:
jingjiayinzi = pd.read_csv('pct1_cal/集合竞价因子.csv')
jingjiayinzi = jingjiayinzi.drop(columns='Unnamed: 0')
df1 = df1.merge(jingjiayinzi, on=['ticker', 'tradeDate'], how='left')

In [ ]:
for f_y in f_y_list:
    result_name = '{}_{}_{}_{}'.format(data_source, model_name, f_y, target_type)
    print(result_name)

    df_result_all = pd.DataFrame()
    total_IC = 0
    start = time.time()
    #epoch_range = range(0, 1)

    for epoch in epoch_range:
        #epoch = num_epoch + 1
        print('----- EPOCH {}------'.format(epoch))
        update_n = epoch * update
        # get a list of train dates
        epoch_t_train = epoch_ts[train_si + update_n : train_ei + update_n]
        # get a list of test dates
        epoch_t_test = epoch_ts[test_si + update_n : test_ei + update_n]
        df_train = df1[df1.tradeDate.apply(lambda x: x in epoch_t_train)].reset_index(drop=True)
        df_test = df1[df1.tradeDate.apply(lambda x: x in epoch_t_test)].reset_index(drop=True)
        print('预测时间：', epoch_t_test)
        print('数据大小：', df_train.shape, df_test.shape)

        # 数据筛选 删除上市100天以内的
        #a = pd.to_datetime(df_train.tradeDate)
        #b = pd.to_datetime(df_train.listData)
        #df_train = df_train[a-b > timediff]
        #a = pd.to_datetime(df_test.tradeDate)
        #b = pd.to_datetime(df_test.listData)
        #df_test = df_test[a-b > timediff]

        # 获得 x
        x_train = df_train[f_x].values
        x_test = df_test[f_x].values
        print('处理后x:', x_train.shape, x_test.shape)

        # 获得y
        y_train = df_train[f_y].values
        y_test = df_test[f_y].values
        #y_train = assign_weight(y_train, weightdic)
        #y_test= assign_weight(y_test, weightdic)
        #y_train['weight'] = 1
        #y_test['weight'] = 1
        print('处理后y:', y_train.shape, y_test.shape)
        #model = XGBRegressor(n_estimators=863, max_depth=6, learning_rate = 0.0142, subsample = 0.68)
        #model = lgb.LGBMRegressor(max_depth=5, num_leaves=10, learning_rate=0.09, n_estimators=100)
        #model = lgb.LGBMRegressor(max_depth=6, num_leaves=25, learning_rate=0.1, n_estimators=100)
        # {'n_estimators': 100, 'learning_rate': 0.01, 'subsample': 0.7, 'max_depth': 5, 'subsample_freq': 5, 'reg_lambda': 0, 'num_leaves': 10, 'min_child_samples': 80, 'max_bin': 20}
        #model = lgb.LGBMRegressor(num_leaves = 15, min_child_samples=200, max_bin = 256, n_estimators=100, \
        #    importance_type='gain', deterministic= True, n_jobs=-1, subsample=0.9, subsample_freq=2)
        # 'n_estimators': 100, 'learning_rate': 0.07519120996144206, 'subsample': 0.7489953327400968, 'max_depth': 6, 'subsample_freq': 8, 'reg_lambda': 50, 'num_leaves': 60
        #model = lgb.LGBMRegressor(num_leaves= 60, n_estimators=100, learning_rate= 0.07519, reg_lambda = 50,
        #subsample= 0.7489953, subsample_freq=8, max_depth=6)
        #model = lgb.LGBMRegressor(**params)
        #model.fit(x_train, y_train, eval_set=[(x_train,y_train), 
        #(x_test,y_test)], eval_metric='l2')
        lgb_train = lgb.Dataset(x_train, y_train)
        model = lgb.train(params,
                        lgb_train,
                        num_boost_round=params['num_boost_round'],
                        callbacks = [lgb.reset_parameter(learning_rate = params['lrlist'])]
                        )
        y_pred = model.predict(x_test)

        # 获得结果
        print(f'耗时:{time.time() - start}')
        print('get result')
        df_result = df_test[f_index].copy()
        df_result['y'] = y_test
        df_result['y_pred'] = y_pred
        IC = df_result[['y', 'y_pred']].corr().iloc[0,1]
        print(IC)
        df_result_all = df_result_all.append(df_result)

    print(f'耗时:{time.time() - start}') 
    print('sort values')
    df_result_all = df_result_all.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
    IC = df_result_all[['y', 'y_pred']].corr().iloc[0,1]
    today = (datetime.datetime.now()).strftime("%Y-%m-%d:%H:%M:%S")
    df_result_all.to_csv('{}/{}/{}{}_train_{}.csv'.format(file_location, file_type, result_name, today, round(IC, 4)), index=False)
    print('======== COMPLETED {} {} ========'.format(model_name, IC))


## Feature Importance

In [ ]:
from lightgbm import plot_importance
plot_importance(model, max_num_features=20)

In [ ]:
f_x[1], f_x[2], f_x[7]

In [ ]:
y_pred = model.predict(x_train)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
parameters = {
     'max_depth': [4, 5, 6, 7],#range(3,8,2), 
     'num_leaves': [10, 30, 60],#range(10, 170, 20), 
     'learning_rate': [0.01, 0.05, 0.1],
     #'min_child_samples': range(10, 210, 20),
     #'subsample': [i/10.0 for i in range(6,10)], 
     #'subsample_freq': range(1,10,1)
}
gbm = lgb.LGBMRegressor( objective = 'regression',
                         learning_rate = 0.05,
                         num_leaves = 40,
                         n_estimators=100,
                         max_depth=6,
                         metrics='rmse',
                         #subsample= 0.9, 
                         #subsample_freq=2,
                         #min_child_samples=150,
                         #max_bin= 256,
                         n_jobs=-1)
# 有了gridsearch我们便不需要fit函数
gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='neg_mean_squared_error',\
     cv=TimeSeriesSplit(n_splits=20, test_size=22), verbose=1, n_jobs=-1)
gsearch.fit(x_train, y_train)

print("Best score: %0.3f" % gsearch.best_score_)
print("Best parameters set:")
best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

## Get Result

In [3]:
def convertFile(filenames, file_location):
    for file in filenames:
        name = file[:-26] + file[-11:-4]
        print(name)
        df = pd.read_csv(file_location + file)
        df.rename(columns = {'tradeDate': 'date', 'y_pred': 'prediction'}, inplace = True)
        df['date'] = df.date.apply(lambda x: str(x).replace('-', ''))
        df = df[['ticker', 'date', 'prediction']]

        dates = df.date.sort_values().unique()
        def run(date):
            print(date)
            temp = df[df.date == date]
            temp = temp.sort_values(by=['prediction'], ascending= False).reset_index(drop=True)
            #temp.to_csv('Predictions/zz500_{}/{}.csv'.format(name, date), index=False)
            temp.to_csv('{}{}/{}.csv'.format(file_location, name, date), index=False)

        print('PARALLEL')
        start = time.time()
        parallel_obj = Parallel(n_jobs=48)(delayed(run)(date) for date in dates)
        print(f'耗时:{time.time() - start}')

In [4]:
filenames = ['full_Alter_extra_221_LGBMRegressor_openclose_pct1_full_r2022-07-31:11:06:20_train_0.1086.csv',
'full_Alter_extra_221_LGBMRegressor_openclose_pct1_full_rank_r2022-07-31:11:38:20_train_0.1734.csv',
'full_Alter_extra_221_LGBMRegressor_openclose_pct1_r2022-07-31:10:37:38_train_0.1606.csv',
'full_Alter_extra_221_LGBMRegressor_openclose_pct1_rank_r2022-07-31:10:07:00_train_0.1752.csv']
file_location = 'pct1_cal/full/'
convertFile(filenames, file_location)

full_Alter_extra_221_LGBMRegressor_openclose_pct1_full_r2022-07-31_0.1086
PARALLEL
20190102
20190103
20190104
20190107
20190108
20190109
20190110
20190111
20190114
20190115
20190116
20190117
20190118
20190121
20190122
20190123
20190124
20190125
20190128
20190129
20190130
20190131
20190201
20190211
20190212
20190213
20190214
20190215
20190218
20190219
20190220
20190221
20190222
20190225
20190226
20190227
20190228
20190301
20190304
20190305
20190306
20190307
20190308
20190311
20190312
20190313
20190314
20190315
20190318
20190319
